# Save and Upload a Model to S3

Save a trained model in ONNX format and upload the file to an S3 bucket.

### 1. Import the required libraries

In [ ]:
import os
import boto3
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.svm import OneClassSVM

from skl2onnx import to_onnx

### 2. Train a sample model

Generate a random data set of normally-distributed data points.

In [ ]:
X, _ = make_blobs(100, centers=1, n_features=2, random_state=0)

Train a model for anomaly detection by using the generated data.

In [ ]:
model = OneClassSVM(nu=0.1)
model.fit(X)

Run the anomaly detection model.

In [56]:
y_pred = model.predict(X)

Plot the results.
The diagram displays anomalies in red.

In [ ]:
colors = np.array([
    "#ef3300",  # Outlier -> Anomaly
    "#00aa52",  # Inlier -> Ok
])
is_inlier = (y_pred + 1) // 2
y_pred_colors = colors[is_inlier]
plt.scatter(X[:, 0], X[:, 1], color=y_pred_colors)

### 3. Export the trained model to ONNX

Use a single sample row for the model to infer the input shape.

In [ ]:
sample_row = X[:1]
sample_row

Convert the Scikit-learn model to ONNX.

In [ ]:
onnx_model = to_onnx(model, sample_row)
onnx_model

Save the ONNX model to a file.

In [ ]:
onnx_file_name = "anomaly_detection.onnx"

with open(onnx_file_name, "wb") as f:
    f.write(onnx_model.SerializeToString())

### 4. Upload the file to the S3 bucket.

Use the `boto3` library with the parameters of the data connection.

> **IMPORTANT**
>
> The `verify=False` parameter ignores the errors caused by the self-signed certificates of the lab environment.
> In production, you should always remove this parameter to verify the SSL certificates of your S3 services.

In [ ]:
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

s3_client = boto3.client(
    "s3",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    verify=False
)

s3_client.upload_file(onnx_file_name, bucket_name, Key=onnx_file_name)

print(f"File {onnx_file_name} uploaded to S3!")

Open https://minio-minio.apps.ocp4.example.com/.

Authenticate with the `minio` access key and the `minio123` secret key.

Verify that the `saved-models` bucket contains the ONNX model file.